In [ ]:
import json
import pandas as pd
import numpy as np
import os
os.chdir(r"D:/Wisconsin/628/project 3/yelp_Fall2023/")

In [ ]:
combine = pd.read_csv('combined_data.csv')

In [ ]:
indianapolis_breakfast = combine

In [ ]:
indianapolis_breakfast = indianapolis_breakfast.drop_duplicates(subset='business_id')

In [ ]:
business = pd.read_json("business.json",lines=True)

In [ ]:
pattern = r'(?i).*\b(breakfast & brunch)\b.*'
breakfast = business[business['categories'].str.contains(pattern, na=False)]

In [ ]:
indianapolis_breakfast1 = breakfast[breakfast['city'] == 'Indianapolis']

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
sse = []
n = 20
for k in range(1, n):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(indianapolis_breakfast1[['longitude', 'latitude']])
    sse.append(kmeans.inertia_)

# 绘制肘部图
plt.figure(figsize=(15, 6))
plt.plot(range(1, n), sse, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

k = 10

# apply k-means
kmeans = KMeans(n_clusters=k, random_state=1202)
kmeans.fit(indianapolis_breakfast[['longitude', 'latitude']])

# add result to data set 
indianapolis_breakfast['cluster'] = kmeans.labels_

In [ ]:
cluster_info = indianapolis_breakfast.groupby('cluster')['stars_x'].agg(['count', 'mean'])

# rename column
cluster_info.rename(columns={'count': 'Number of Restaurants', 'mean': 'Average Stars'}, inplace=True)

# order by average star
cluster_info_sorted = cluster_info.sort_values(by='Average Stars', ascending=False)
cluster_avg_stars = indianapolis_breakfast.groupby('cluster')['stars_x'].mean().sort_values(ascending=False)
sorted_cluster_names = {old: new + 1 for new, old in enumerate(cluster_avg_stars.index)}

# rename cluster
indianapolis_breakfast['cluster'] = indianapolis_breakfast['cluster'].map(sorted_cluster_names)
cluster_stars_mean = indianapolis_breakfast.groupby('cluster')['stars_x'].mean()

# get average star in each cluster
print(cluster_stars_mean)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import folium
import numpy as np
import matplotlib.pyplot as plt
from branca.element import Template, MacroElement

map = folium.Map(location=[39.7684, -86.1581], zoom_start=12)
# plot for each cluster
colors = ['red', 'blue', 'green', 'purple', 'orange']
for cluster_num in range(1,k+1):
    cluster_data = indianapolis_breakfast[indianapolis_breakfast['cluster'] == cluster_num]
    center = cluster_data[['latitude', 'longitude']].mean().values
    # compute radius
    std_dev = np.mean(cluster_data[['latitude', 'longitude']].std().values)
    radius = std_dev * 150000 
    folium.Circle(
        location=center,
        radius=radius,
        color=colors[cluster_num % len(colors)],
        fill=True,
        fill_opacity=0.4
    ).add_to(map)
    # add mark based on stars
    for index, row in cluster_data.iterrows():
        if row['stars_x'] >= 4:
            marker_color = 'darkgreen'
        elif row['stars_x'] >= 3:
            marker_color = 'lightgreen'
        elif row['stars_x'] >= 2:
            marker_color = 'orange'
        else:
            marker_color = 'red'
        
        folium.Marker(
            [row['latitude'], row['longitude']],
            icon=folium.Icon(color=marker_color, icon='star_x')
        ).add_to(map)
    i = 1
    folium.Marker(
        location=center,
        popup=f'Cluster {cluster_num} (Avg Stars: {cluster_data["stars_x"].mean():.2f})',
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map)
    i = i+1
    
    
template = """
{% macro html(this, kwargs) %}
<div style ="position: fixed; 
             top: 50px; left: 50px; width: 150px; height: 150px; 
             border:2px solid grey; z-index:9999; font-size:14px;
             ">&nbsp; Stars Legend <br>
  &nbsp; 4-5 stars &nbsp; <i class="fa fa-map-marker fa-2x" style="color:darkgreen"></i><br>
  &nbsp; 3-4 stars &nbsp; <i class="fa fa-map-marker fa-2x" style="color:lightgreen"></i><br>
  &nbsp; 2-3 stars &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i><br>
  &nbsp; 1-2 stars &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>
</div>
{% endmacro %}
"""

macro = MacroElement()
macro._template = Template(template)

map.get_root().add_child(macro)

# save map
map.save('indianapolis_breakfast_clusters_with_stars.html')

In [ ]:
# check zip code
zip_clusters = indianapolis_breakfast.groupby('ZIP Code')['cluster'].unique()

for zip_code, clusters in zip_clusters.items():
    if len(clusters) > 1:
        selected_cluster = np.random.choice(clusters)
        indianapolis_breakfast.loc[indianapolis_breakfast['ZIP Code'] == zip_code, 'cluster'] = selected_cluster

In [ ]:
# combine two data sets
business_ids = indianapolis_breakfast['business_id'].unique()
matching_rows = indianapolis_breakfast1[indianapolis_breakfast1['business_id'].isin(business_ids)]

In [ ]:
indianapolis_breakfast1 = indianapolis_breakfast1.merge(indianapolis_breakfast[['business_id', 'cluster','ZIP Code']], 
                                                        on='business_id', 
                                                        how='inner')

In [ ]:
import pandas as pd

# deal with attributes column
def process_element(element):
    new_element = {}
    if element is None:
        return new_element

    for key, value in element.items():
        if isinstance(value, str) and value.startswith("{"):
            try:
                sub_dict = eval(value)
            except:
                sub_dict = {}
            for sub_key, sub_value in sub_dict.items():
                new_key = f"{key}_{sub_key}"
                new_element[new_key] = 1 if sub_value == 'True' else 0 if sub_value == 'False' else sub_value
        else:
            new_element[key] = 1 if value == 'True' else 0 if value == 'False' else value
    return new_element

expanded_data = indianapolis_breakfast1['attributes'].apply(process_element)
expanded_df = pd.DataFrame(expanded_data.tolist())

In [ ]:
df = expanded_df

In [ ]:
# delete useless information
def remove_u_prefix(text):
    if isinstance(text, str):
        return text.replace("u'", "'", 1)  
    return text 


for column in df.columns:
    df[column] = df[column].apply(remove_u_prefix)

In [ ]:
expanded_df = df

In [ ]:
expanded_df.head()

In [ ]:
import pandas as pd

# one-hot encoding
expanded_df = pd.get_dummies(expanded_df)

In [ ]:
expanded_df.reset_index(drop=True, inplace=True)
indianapolis_breakfast1.reset_index(drop=True, inplace=True)
indianapolis_breakfast.reset_index(drop=True, inplace=True)

In [ ]:
indianapolis_breakfast1.to_csv('indianapolis_zipcode_cluster.csv', index=True)

In [ ]:
import pandas as pd
# See stars of attributes
n = 10

cluster_1_rows = indianapolis_breakfast1['cluster'] <12
print(indianapolis_breakfast[indianapolis_breakfast['cluster'] == n]['ZIP Code'].unique())
mean_stars_cluster_1 = indianapolis_breakfast1[cluster_1_rows]['stars'].mean()
print(f"Average star of cluster {n}:{mean_stars_cluster_1}")

above_avg_results = {}
below_avg_results = {}

for col in expanded_df.columns:
    valid_rows = (expanded_df[col] == 1) & cluster_1_rows
    valid_stars = indianapolis_breakfast1[valid_rows]['stars']
    avg_value = valid_stars.mean() if not valid_stars.empty else None

    if avg_value is not None:
        if avg_value > mean_stars_cluster_1:
            above_avg_results[col] = avg_value
        elif avg_value < mean_stars_cluster_1:
            below_avg_results[col] = avg_value

# order result
above_avg_results_sorted = dict(sorted(above_avg_results.items(), key=lambda item: item[1], reverse=True))
below_avg_results_sorted = dict(sorted(below_avg_results.items(), key=lambda item: item[1]))

print("Tags with Average Stars Above Cluster {n} Mean:")
print(above_avg_results_sorted)
print("\nTags with Average Stars Below Cluster {n} Mean:")
print(below_avg_results_sorted)
